In [1]:
import pandas as pd


data_ls = []
name=['EN/EN','AL/AL','CN/CN','SG/SG']

for i in name:
    file = "./"+i+"/train"
    file_object = open(file, "r",encoding='UTF-8')
    dataset=[]#its a list
    for line in file_object:
        dataset.append(line)#lines are string
        
    data=[]
    #print(type(dataset[0]))
    for i in range(len(dataset)):
        temp = dataset[i].split()
        if (len(temp)==2):
            data.append(temp)# its a list of lists with word and tag
    data_ls.append(pd.DataFrame(data, columns = ['Word', 'Tag']))

EN = data_ls[0]
AL = data_ls[1]
CN = data_ls[2]
SG = data_ls[3]

In [2]:

for i in range(len(data_ls)):
    print(name[i])
    print(data_ls[i].head())
    print("\n")

EN/EN
        Word     Tag
0  Municipal    B-NP
1      bonds    I-NP
2        are    B-VP
3  generally  B-ADVP
4          a  B-ADJP


AL/AL
  Word         Tag
0    小  B-DISTRICT
1    港  I-DISTRICT
2    四       B-POI
3    方       I-POI
4    家       I-POI


CN/CN
  Word Tag
0    这   O
1   大概   O
2    是   O
3   所有   O
4   版本   O


SG/SG
    Word         Tag
0    I'm           O
1  about           O
2     to           O
3    see           O
4   Adam  B-positive




In [3]:
for i in range(len(data_ls)):
    print(name[i])
    print(data_ls[i]['Tag'].value_counts())
    print("\n")

EN/EN
I-NP       54591
B-NP       47305
O          23872
B-PP       18387
B-VP       18261
I-VP       10159
B-ADVP      3565
B-SBAR      1899
B-ADJP      1751
I-ADJP       574
B-PRT        468
I-ADVP       363
I-PP         223
I-CONJP       64
B-CONJP       49
I-SBAR        48
B-INTJ        26
B-LST         11
I-INTJ         7
I-UCP          4
B-UCP          1
Name: Tag, dtype: int64


AL/AL
I-POI          28472
I-ROAD         15178
I-DISTRICT     13364
I-TOWN         10748
I-CITY         10430
I-PROV          8666
B-POI           7348
B-DISTRICT      6856
B-ROAD          6324
B-CITY          5697
B-ROADNO        5061
I-ROADNO        4666
B-TOWN          4612
I-SUBPOI        4557
B-PROV          4488
B-REDUNDANT     4137
I-REDUNDANT     3668
B-HOUSENO       3455
I-COMMUNITY     3148
B-ROOMNO        3107
I-HOUSENO       3045
I-PERSON        2232
I-DEVZONE       1834
B-SUBPOI        1650
I-CELLNO        1488
B-COMMUNITY     1479
B-CELLNO        1326
I-FLOORNO       1237
B-FLOORNO       1

In [4]:
def emission(df,word,tag):
    is_tag = (df.Tag==tag)
    y = is_tag.sum()
    xy = (df.Word[is_tag]==word).sum()
    return xy/y

In [5]:
print(emission(EN,"are","B-VP"))

0.03707354471277586


In [6]:
def smoothen(df,k):
    ls = df['Word'].value_counts()[df['Word'].value_counts()<k].index.tolist()
    df.loc[df['Word'].isin(ls), 'Word'] = "#UNK#"
    return df

EN=smoothen(EN,k=3)
AL=smoothen(AL,k=3)
CN=smoothen(CN,k=3)
SG=smoothen(SG,k=3)

In [7]:
for i in range(len(data_ls)):
    print(name[i])
    print(data_ls[i].head(20))
    print("\n")

EN/EN
         Word     Tag
0       #UNK#    B-NP
1       bonds    I-NP
2         are    B-VP
3   generally  B-ADVP
4           a  B-ADJP
5         bit  I-ADJP
6       #UNK#  I-ADJP
7        than    B-PP
8   corporate    B-NP
9       bonds    I-NP
10         in    B-PP
11          a    B-NP
12  recession    I-NP
13          ,       O
14        but       O
15        not  B-ADJP
16         as  I-ADJP
17       safe  I-ADJP
18         as    B-PP
19      bonds    B-NP


AL/AL
    Word         Tag
0      小  B-DISTRICT
1      港  I-DISTRICT
2      四       B-POI
3      方       I-POI
4      家       I-POI
5      园       I-POI
6     17   B-HOUSENO
7      幢   I-HOUSENO
8   1363    B-CELLNO
9      室    I-CELLNO
10     嘉      B-CITY
11     兴      I-CITY
12     南  B-DISTRICT
13     湖  I-DISTRICT
14     区  I-DISTRICT
15     文      B-ROAD
16     昌      I-ROAD
17     路      I-ROAD
18     新       B-POI
19     文       I-POI


CN/CN
     Word        Tag
0       这          O
1      大概          O
2       是   

In [8]:
from tqdm.notebook import tqdm
tqdm.pandas()


EN_count_series = EN.groupby(['Word', 'Tag']).size()
AL_count_series = AL.groupby(['Word', 'Tag']).size()
CN_count_series = CN.groupby(['Word', 'Tag']).size()
SG_count_series = SG.groupby(['Word', 'Tag']).size()

EN_tag_count = EN['Tag'].value_counts()
AL_tag_count = AL['Tag'].value_counts()
CN_tag_count = CN['Tag'].value_counts()
SG_tag_count = SG['Tag'].value_counts()

EN_count_u = pd.Series(EN_count_series.index.get_level_values('Tag').tolist(), index=EN_count_series.index).progress_apply(lambda x: EN_tag_count[x])
AL_count_u = pd.Series(AL_count_series.index.get_level_values('Tag').tolist(), index=AL_count_series.index).progress_apply(lambda x: AL_tag_count[x])
CN_count_u = pd.Series(CN_count_series.index.get_level_values('Tag').tolist(), index=CN_count_series.index).progress_apply(lambda x: CN_tag_count[x])
SG_count_u = pd.Series(SG_count_series.index.get_level_values('Tag').tolist(), index=SG_count_series.index).progress_apply(lambda x: SG_tag_count[x])

EN_count_series = EN_count_series / EN_count_u
AL_count_series = AL_count_series / AL_count_u
CN_count_series = CN_count_series / CN_count_u
SG_count_series = SG_count_series / SG_count_u


count_series=[EN_count_series, AL_count_series, CN_count_series, SG_count_series]

In [9]:
for i in range(len(count_series)):
    print(name[i])
    print(count_series[i])
    print("\n")

EN/EN
Word          Tag   
!             O         0.000628
#             B-ADJP    0.001713
              B-NP      0.000423
              I-NP      0.000183
              O         0.000042
#UNK#         B-ADJP    0.155340
              B-ADVP    0.062272
              B-INTJ    0.346154
              B-LST     0.090909
              B-NP      0.082507
              B-PP      0.002502
              B-SBAR    0.001580
              B-VP      0.077980
              I-ADJP    0.229965
              I-ADVP    0.082645
              I-INTJ    0.285714
              I-NP      0.138832
              I-UCP     0.250000
              I-VP      0.104046
              O         0.002346
$             B-ADJP    0.005711
              B-NP      0.022957
              I-NP      0.006924
              O         0.000084
%             I-ADJP    0.027875
              I-ADVP    0.002755
              I-NP      0.018556
&             I-NP      0.003077
              O         0.000126
'             B-

In [10]:
# def argmax(df,word):
#     if (word==""):
#         return ""
#     tags = (df.Tag[df.Word==word]).value_counts().index.tolist()
#     if (len(tags)==0):
#         tags = (df.Tag[df.Word=="#UNK#"]).value_counts().index.tolist()
#     emission_ls=[]
#     for i in tags:
#         emission_ls.append(emission(df,word,i))
#     return(tags[emission_ls.index(max(emission_ls))])

def argmax(df,word):
    if (word==""):
        return ""
    try:
        value = df.loc[word].idxmax()
    except:
        value = df.loc["#UNK#"].idxmax()
    
    return (value)

# def argmax(df,word):
#     if (word==""):
#         return ""
#     tags = pd.DataFrame((df.Tag[df.Word==word]).value_counts().index.tolist(), columns=['Tags'])
#     if (len(tags.index)==0):
#         tags = pd.DataFrame((df.Tag[df.Word=="#UNK#"]).value_counts().index.tolist(), columns=['Tags'])
#         word="#UNK#"
#     tags['Emission'] = tags['Tags'].apply(lambda x: emission(df,word,x))
#     return(tags['Tags'].loc[tags['Emission'].idxmax()])

In [11]:
test_ls = []
count=0
for i in name:
    file = "./"+i+"/dev.in"
    file_object = open(file, "r",encoding='UTF-8')
    ls=[]
    for line in file_object:
        ls.append(line.strip())
    df = pd.DataFrame(ls, columns = ['Word'])
    df['Tag'] = df['Word'].progress_apply(lambda x: argmax(count_series[count], x))
    test_ls.append(df)
    count+=1

EN_test = test_ls[0]
AL_test = test_ls[1]
CN_test = test_ls[2]
SG_test = test_ls[3]

In [12]:
for i in range(len(test_ls)):
    print(name[i])
    print(test_ls[i])
    print("\n")
    

EN/EN
              Word      Tag
0              HBO     B-NP
1              has     B-VP
2            close   B-ADJP
3               to     B-PP
4               24     I-NP
5          million     I-NP
6      subscribers     I-NP
7               to     B-PP
8              its     B-NP
9              HBO     B-NP
10             and    I-UCP
11         Cinemax   B-INTJ
12        networks     I-NP
13               ,        O
14           while   B-SBAR
15        Showtime     B-NP
16             and    I-UCP
17             its     B-NP
18          sister   B-INTJ
19         service     I-NP
20               ,        O
21             The     B-NP
22           Movie   B-INTJ
23         Channel   B-INTJ
24               ,        O
25            have     I-VP
26            only  I-CONJP
27           about     B-PP
28              10   I-ADVP
29         million     I-NP
...            ...      ...
27195           is   I-INTJ
27196          due   B-ADJP
27197         only  I-CONJP
27198       pa

In [14]:

test_out_ls=[]
for i in name:
    file = "./"+i+"/dev.out"
    file_object = open(file, "r",encoding='UTF-8')
    ls=[]
    for line in file_object:
        temp=line.split()
        if (len(temp)==0):
            temp.append("")
            temp.append("")
        ls.append(temp)
    test_out_ls.append(pd.DataFrame(ls, columns = ['Word', 'Tag']))

EN_test_out = test_out_ls[0]
AL_test_out = test_out_ls[1]
CN_test_out = test_out_ls[2]
SG_test_out = test_out_ls[3]

In [15]:

for i in range(len(test_out_ls)):
    print(name[i])
    print(test_out_ls[i])
    print("\n")

EN/EN
              Word     Tag
0              HBO    B-NP
1              has    B-VP
2            close    B-NP
3               to    I-NP
4               24    I-NP
5          million    I-NP
6      subscribers    I-NP
7               to    B-PP
8              its    B-NP
9              HBO    I-NP
10             and    I-NP
11         Cinemax    I-NP
12        networks    I-NP
13               ,       O
14           while  B-SBAR
15        Showtime    B-NP
16             and       O
17             its    B-NP
18          sister    I-NP
19         service    I-NP
20               ,       O
21             The    B-NP
22           Movie    I-NP
23         Channel    I-NP
24               ,       O
25            have    B-VP
26            only    B-NP
27           about    I-NP
28              10    I-NP
29         million    I-NP
...            ...     ...
27195           is    B-VP
27196          due  B-ADJP
27197         only  I-ADJP
27198       partly  I-ADJP
27199           to    

In [16]:
for i in range(len(name)):
    file = "./"+name[i]+"/dev.p2.out"
    test_ls[i].to_csv(file, sep=" ", index=False, header=False)

In [22]:
!py ./EvalScript/evalResult.py ./EN/EN/dev.out ./EN/EN/dev.p2.out


#Entity in gold data: 13179
#Entity in prediction: 19406

#Correct Entity : 9152
Entity  precision: 0.4716
Entity  recall: 0.6944
Entity  F: 0.5617

#Correct Sentiment : 7644
Sentiment  precision: 0.3939
Sentiment  recall: 0.5800
Sentiment  F: 0.4692
